<a href="https://colab.research.google.com/github/fatemafaria142/Large-Language-Models-and-Vector-Databases-for-Retrieval-Augmented-Generation/blob/main/Text_Documents_FAISS_retriever_using_all_MiniLM_L6_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Install Required Packages**

In [1]:
!pip install accelerate peft bitsandbytes transformers trl datasets torch langchain -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 51.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.9/815.9 kB 46.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 80.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━

In [3]:
!pip install faiss-cpu -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 47.3 MB/s eta 0:00:00


In [4]:
!pip install sentence-transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 970.7 kB/s eta 0:00:00


### **Embedding Model**
* https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2

In [5]:
from langchain.embeddings import HuggingFaceEmbeddings
model_name = "sentence-transformers/all-MiniLM-L6-v2"
embedding = HuggingFaceEmbeddings(model_name=model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

### **Import Necessary Libraries**

In [10]:
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.storage import InMemoryByteStore
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader

### **Access Dataset**

In [7]:
!wget https://github.com/Shafi2016/Youtube/raw/main/stock_market_june_2023.zip -O stock_market_june_2023.zip
!unzip stock_market_june_2023.zip

--2024-02-17 04:29:54--  https://github.com/Shafi2016/Youtube/raw/main/stock_market_june_2023.zip
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Shafi2016/Youtube/main/stock_market_june_2023.zip [following]
--2024-02-17 04:29:54--  https://raw.githubusercontent.com/Shafi2016/Youtube/main/stock_market_june_2023.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10638 (10K) [application/zip]
Saving to: ‘stock_market_june_2023.zip’

stock_market_june_2 100%[===================>]  10.39K  --.-KB/s    in 0s      

2024-02-17 04:29:54 (97.6 MB/s) - ‘stock_market_june_2023.zip’ saved [10638/10638]

A

### **Documentation Link:** https://python.langchain.com/docs/modules/data_connection/retrievers/vectorstore

In [8]:
# Initializes a DirectoryLoader to load text files from the specified directory.
loader = DirectoryLoader('./stock_market_june_2023/', glob="./*.txt", loader_cls=TextLoader)

# Loads all the documents present in the directory.
raw_documents = loader.load()


In [11]:
# Initialize a RecursiveCharacterTextSplitter with a specified chunk size and overlap.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)

# Split the loaded documents into chunks of text using the defined text_splitter.
documents = text_splitter.split_documents(raw_documents)

# Check the length of the resulting text chunks.
print("Number of text chunks:", len(documents))

# Load the document, split it into chunks, embed each chunk and load it into the vector store.
db = FAISS.from_documents(documents, embedding)

Number of text chunks: 49


### **Similarity search**

In [12]:
query = "How has Shopify adapted to the changing business landscape, and what significant changes has it implemented recently, such as layoffs and selling its logistics business?"
docs = db.similarity_search(query)
print(docs[0].page_content)

To adapt to the changing landscape, Shopify implemented significant changes, including workforce layoffs of approximately 20% and selling its logistics business to Flexport. Although these changes incur near-term expenses, such as restructuring and impairment charges, they will ultimately reduce overhead costs and make Shopify more asset-light in the long run.


### **Similarity search by vector**
* It is also possible to do a search for documents similar to a given embedding vector using `similarity_search_by_vector` which accepts an embedding vector as a parameter instead of a string.

In [13]:
embedding_vector = embedding.embed_query(query)
docs = db.similarity_search_by_vector(embedding_vector)
print(docs[0].page_content)

To adapt to the changing landscape, Shopify implemented significant changes, including workforce layoffs of approximately 20% and selling its logistics business to Flexport. Although these changes incur near-term expenses, such as restructuring and impairment charges, they will ultimately reduce overhead costs and make Shopify more asset-light in the long run.


### **Maximum marginal relevance retrieval**

In [14]:
retriever = db.as_retriever(search_type="mmr")
response = retriever.get_relevant_documents(query)
print(response)

[Document(page_content='To adapt to the changing landscape, Shopify implemented significant changes, including workforce layoffs of approximately 20% and selling its logistics business to Flexport. Although these changes incur near-term expenses, such as restructuring and impairment charges, they will ultimately reduce overhead costs and make Shopify more asset-light in the long run.', metadata={'source': 'stock_market_june_2023/Shopify.txt'}), Document(page_content='Furthermore, Shopify Audiences, a marketing software utilizing machine learning, allows creating and exporting customer lists to platforms like Pinterest, Facebook, Instagram, and Google for targeted ad campaigns. The adoption of Shopify Audiences by merchants upgrading to Shopify Plus presents a promising growth opportunity.\n\nShopify’s ongoing commitment to innovation and expansion into AI and upmarket segments positions it for continued success and growth.', metadata={'source': 'stock_market_june_2023/Shopify.txt'}), D

In [15]:
for i in range(3):  # Adjust the range based on the number of chunks you want to print
    print(f"{i + 1} no. document: {response[i].page_content}\n")
    print(f"{i + 1} no. source: {response[i].metadata.get('source', 'N/A')}\n")


1 no. document: To adapt to the changing landscape, Shopify implemented significant changes, including workforce layoffs of approximately 20% and selling its logistics business to Flexport. Although these changes incur near-term expenses, such as restructuring and impairment charges, they will ultimately reduce overhead costs and make Shopify more asset-light in the long run.

1 no. source: stock_market_june_2023/Shopify.txt

2 no. document: Furthermore, Shopify Audiences, a marketing software utilizing machine learning, allows creating and exporting customer lists to platforms like Pinterest, Facebook, Instagram, and Google for targeted ad campaigns. The adoption of Shopify Audiences by merchants upgrading to Shopify Plus presents a promising growth opportunity.

Shopify’s ongoing commitment to innovation and expansion into AI and upmarket segments positions it for continued success and growth.

2 no. source: stock_market_june_2023/Shopify.txt

3 no. document: Shopify (NYSE: SHOP) has

### **Similarity score threshold retrieval**

In [16]:
retriever = db.as_retriever(
    search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.5}
)
response = retriever.get_relevant_documents(query)
print(response)

[Document(page_content='To adapt to the changing landscape, Shopify implemented significant changes, including workforce layoffs of approximately 20% and selling its logistics business to Flexport. Although these changes incur near-term expenses, such as restructuring and impairment charges, they will ultimately reduce overhead costs and make Shopify more asset-light in the long run.', metadata={'source': 'stock_market_june_2023/Shopify.txt'}), Document(page_content='In the seven years since its initial public offering, Shopify has been profitable in five, a commendable track record in an industry known for widespread unprofitability. The company has weathered economic volatility, including the challenges posed by the pandemic, by streamlining its focus on delivering market-leading software and solutions to empower its merchant clients.', metadata={'source': 'stock_market_june_2023/Shopify.txt'}), Document(page_content='Shopify (NYSE: SHOP) has 55% increase in its shares so far in 2023

In [17]:
print(len(response))

4


In [18]:
for item in response:
    print(f"document: {item.page_content}\n")
    print(f"source: {item.metadata.get('source', 'N/A')}\n")


document: To adapt to the changing landscape, Shopify implemented significant changes, including workforce layoffs of approximately 20% and selling its logistics business to Flexport. Although these changes incur near-term expenses, such as restructuring and impairment charges, they will ultimately reduce overhead costs and make Shopify more asset-light in the long run.

source: stock_market_june_2023/Shopify.txt

document: In the seven years since its initial public offering, Shopify has been profitable in five, a commendable track record in an industry known for widespread unprofitability. The company has weathered economic volatility, including the challenges posed by the pandemic, by streamlining its focus on delivering market-leading software and solutions to empower its merchant clients.

source: stock_market_june_2023/Shopify.txt

document: Shopify (NYSE: SHOP) has 55% increase in its shares so far in 2023. Despite ongoing price fluctuations, Shopify’s business and growth trajec

### **Specifying top k**

In [19]:
retriever = db.as_retriever(search_kwargs={"k": 1})
response = retriever.get_relevant_documents(query)
print(response)

[Document(page_content='To adapt to the changing landscape, Shopify implemented significant changes, including workforce layoffs of approximately 20% and selling its logistics business to Flexport. Although these changes incur near-term expenses, such as restructuring and impairment charges, they will ultimately reduce overhead costs and make Shopify more asset-light in the long run.', metadata={'source': 'stock_market_june_2023/Shopify.txt'})]


In [20]:
for item in response:
    print(f"document: {item.page_content}\n")
    print(f"source: {item.metadata.get('source', 'N/A')}\n")


document: To adapt to the changing landscape, Shopify implemented significant changes, including workforce layoffs of approximately 20% and selling its logistics business to Flexport. Although these changes incur near-term expenses, such as restructuring and impairment charges, they will ultimately reduce overhead costs and make Shopify more asset-light in the long run.

source: stock_market_june_2023/Shopify.txt



### **Initializing the Model**
* Load the model using a 4-bit configuration, employing double quantization, and set bfloat16 as the compute data type.

* Notably, we opt for the instruct-tuned model in this instance rather than the base model. It's worth mentioning that fine-tuning a base model necessitates a more substantial amount of data!

In [22]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)


bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)


In [23]:
# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

* **Model and Tokenizer:** https://huggingface.co/NousResearch/Llama-2-7b-chat-hf

In [24]:
mode_id = "NousResearch/Llama-2-7b-chat-hf"

In [25]:
model = AutoModelForCausalLM.from_pretrained(
        mode_id, quantization_config=bnb_config, device_map="auto", use_cache=False
    )

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [26]:
tokenizer = AutoTokenizer.from_pretrained("NousResearch/Llama-2-7b-chat-hf")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

### **Text Generation Pipeline**

In [27]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import pipeline

# Additional configuration parameters
do_sample = True
pad_token_id = tokenizer.eos_token_id
top_p = 0.8  # You can adjust this value based on your preference
temperature = 0.6  # You can adjust this value based on your preference

pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512, do_sample=do_sample, pad_token_id=pad_token_id, top_p=top_p, temperature=temperature)
llm = HuggingFacePipeline(pipeline=pipeline)

## **RetrievalQA - Stuff Chain**

* Chain Types Explanation : https://www.comet.com/site/blog/mastering-document-chains-in-langchain/
* https://python.langchain.com/docs/expression_language/interface#invoke

* search_type: "similarity" or "mmr"
* Chain Type: "stuff", "map reduce", "refine", "map_rerank"

* **Stuff Chain:** One way to provide context to a language model is through the stuffing method. This involves putting all relevant data into the prompt for the LangChain's StuffDocumentsChain to process.
The advantage of this method is that it only requires one call to the LLM, and the model has access to all the information at once. However, one downside is that most LLMs can only handle a certain amount of context. For large or multiple documents, stuffing may result in a prompt that exceeds the context limit. Additionally, this method is only suitable for smaller amounts of data. When working with larger amounts, alternative approaches should be used.
* **Refine Chain:** The Refine Documents Chain uses an iterative process to generate a response by analyzing each input document and updating its answer accordingly. It passes all non-document inputs, the current document, and the latest intermediate answer to an LLM chain to obtain a new answer for each document. This chain is ideal for tasks that involve analyzing more documents than can fit in the model's context, as it only passes a single document to the LLM at a time. However, this also means it makes significantly more LLM calls than other chains, such as the Stuff Documents Chain. It may perform poorly for tasks that require cross-referencing between documents or detailed information from multiple documents. The Refine Documents Chain starts with an initial prompt on the first data set and generates output accordingly. The remaining documents pass in the previous output along with the next document, and ask the LLM to refine the output based on the new document. Pros of this method include incorporating more relevant context and potentially less data loss than the MapReduce Documents Chain. However, it requires many more LLM calls and the calls are not independent, meaning they cannot be paralleled like the MapReduce Documents Chain.



In [35]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
# Initializes a retriever object from the vectordb with a specified number of search results (k=3).
# The vectordb.as_retriever() function is likely converting the Chroma object, which contains vectorized representations of text, into a retriever object.
# The retriever can be used to find the most similar vectors in the database given a query vector.
# The search_kwargs={"k": 3} argument suggests that the retriever will return the top 3 most similar vectors for each query.
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k":2})

# create the chain to answer questions
# Initializes a RetrievalQA object with the specified HuggingFacePipeline (llm), retriever, and chain type.
# The return_source_documents parameter set to True means the original source documents will be included in the returned results.

# Create a sample prompt template
template = """[INST] <>\nImagine you are a financial advisor, and a client seeks your guidance. Please provide detailed and accurate answers on "{context}" considering the current economic conditions and market trends. Your response should be comprehensive, well-informed, and tailored to the client's needs. [\INST] """
prompt = PromptTemplate(template=template, input_variables=["question"])

chain = RetrievalQA.from_chain_type(llm=llm,
                                  retriever=retriever,
                                  return_source_documents=True,
                                  chain_type="stuff",
                                  chain_type_kwargs={"prompt": prompt})



question = "How has Shopify adapted to the changing business landscape, and what significant changes has it implemented recently, such as layoffs and selling its logistics business?"
llm_response = chain.invoke({"query": question})

 # This function processes the response from a Language Model (llm) and prints the result and sources.
def process_llm_response(llm_response):
    # Prints the 'result' field from the response, which likely contains the answer from the language model.
    print(llm_response['result'])

    print('\n\nSources:')
    # Iterates over the 'source_documents' field in the response, which contains the documents where the answer was found.
    for source in llm_response["source_documents"]:
        # Prints the 'source' metadata from each document.
        print(source.metadata['source'])

process_llm_response(llm_response)

 As a financial advisor, I appreciate the client's interest in Shopify's recent changes and their potential impact on the company's financial performance. To provide a comprehensive analysis, I will break down the key points and offer insights based on my understanding of the current economic conditions and market trends.

Firstly, it is important to acknowledge that Shopify's decision to lay off approximately 20% of its workforce and sell its logistics business to Flexport is a strategic move to adapt to the changing landscape. By streamlining its operations, Shopify aims to reduce overhead costs and become more asset-light in the long run. Although these changes may incur near-term expenses, such as restructuring and impairment charges, the company's focus on delivering market-leading software and solutions to empower its merchant clients will remain unwavering.

In the current economic climate, Shopify's decision to focus on software and solutions is particularly well-timed. As the 

## **RetrievalQA - Map Reduce Chain**
* **Map Reduce Chain:** To process large amounts of data efficiently, the MapReduceDocumentsChain method is used. This involves applying an LLM chain to each document individually (in the Map step), producing a new document. Then, all the new documents are passed to a separate combine documents chain to get a single output (in the Reduce step). If necessary, the mapped documents can be compressed before passing them to the combine documents chain. This compression step is performed recursively. This method requires an initial prompt on each chunk of data. For summarization tasks, this could be a summary of that chunk, while for question-answering tasks, it could be an answer based solely on that chunk. Then, a different prompt is run to combine all the initial outputs. The pros of this method are that it can scale to larger documents and handle more documents than the StuffDocumentsChain. Additionally, the calls to the LLM on individual documents are independent and can be parallelized. The cons are that it requires many more calls to the LLM than the StuffDocumentsChain and loses some information during the final combining call.

In [36]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

combine_template = "Write a summary of the following text:\n\n{summaries}"
combine_prompt_template = PromptTemplate.from_template(
    template=combine_template)

question_template = """
[INST] <>\nImagine you are a financial advisor, and a client seeks your guidance. Please provide detailed and accurate answers on {context} considering the current economic conditions and market trends. Your response should be comprehensive, well-informed, and tailored to the client's needs. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer.[\INST]
Question: {question}
Helpful Answer:"""
question_prompt_template = PromptTemplate.from_template(template=question_template)


# create retriever chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=db.as_retriever(
        search_kwargs={'fetch_k': 4, 'k': 3}, search_type='mmr'),
    return_source_documents=True,
    chain_type="map_reduce",
    chain_type_kwargs={"question_prompt": question_prompt_template,
                       "combine_prompt": combine_prompt_template}
)

question = "How has Shopify adapted to the changing business landscape, and what significant changes has it implemented recently, such as layoffs and selling its logistics business?"
llm_response = chain.invoke({"query": question})

 # This function processes the response from a Language Model (llm) and prints the result and sources.
def process_llm_response(llm_response):
    # Prints the 'result' field from the response, which likely contains the answer from the language model.
    print(llm_response['result'])

    print('\n\nSources:')
    # Iterates over the 'source_documents' field in the response, which contains the documents where the answer was found.
    for source in llm_response["source_documents"]:
        # Prints the 'source' metadata from each document.
        print(source.metadata['source'])

process_llm_response(llm_response)

 As a financial advisor, I appreciate the client's interest in Shopify's recent changes and their potential impact on the company's financial performance. To provide a comprehensive analysis, I will address the client's questions and offer insights based on my understanding of the current economic conditions and market trends.

Firstly, the client's observation that Shopify implemented significant changes, including workforce layoffs of approximately 20% and selling its logistics business to Flexport, is accurate. These changes were aimed at adapting to the changing landscape of the e-commerce industry, where competition is increasing and consumer preferences are evolving rapidly. While these changes may incur near-term expenses, such as restructuring and impairment charges, they are expected to reduce overhead costs and make Shopify more asset-light in the long run.

In the current economic environment, it is crucial for companies like Shopify to be agile and adaptable to stay competi